# Shanghai Weather Dataset

## load full dataset

In [60]:
import pandas as pd
from parallel_pandas import ParallelPandas

ParallelPandas.initialize()

In [61]:
df_raw = pd.read_csv('archive/Shanghai AQI and Wheather 2014-2021.csv')

In [62]:
df_raw

,date,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,DewPointC,FeelsLikeC,HeatIndexC,...,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,AQI,AQI_Explained
0,2014-01-01,15.0,5.0,0.0,8.7,4.0,0.0,-1.0,11.0,12.0,...,0.0,43.0,0.0,1021.0,15.0,10.0,242.0,12.0,319.0,Hazardous
1,2014-01-02,14.0,7.0,0.0,8.7,4.0,2.0,4.0,11.0,13.0,...,3.0,60.0,0.0,1019.0,14.0,10.0,141.0,14.0,352.0,Hazardous
2,2014-01-03,16.0,9.0,0.0,8.7,4.0,10.0,3.0,11.0,12.0,...,26.0,55.0,0.0,1017.0,16.0,10.0,295.0,14.0,338.0,Hazardous
3,2014-01-04,10.0,4.0,0.0,5.5,2.0,17.0,3.0,7.0,9.0,...,24.0,68.0,0.1,1022.0,10.0,10.0,169.0,14.0,355.0,Hazardous
4,2014-01-05,10.0,3.0,0.0,8.7,3.0,24.0,3.0,9.0,9.0,...,12.0,66.0,0.0,1024.0,10.0,10.0,117.0,6.0,343.0,Hazardous
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2497,2021-01-26,12.0,10.0,0.0,4.0,3.0,88.0,7.0,10.0,11.0,...,91.0,77.0,1.1,1023.0,12.0,8.0,233.0,9.0,229.0,Very unhealthy
2498,2021-01-27,11.0,7.0,0.0,8.7,2.0,96.0,4.0,7.0,9.0,...,55.0,72.0,0.0,1024.0,11.0,10.0,49.0,11.0,200.0,Unhealthy
2499,2021-01-28,10.0,4.0,0.0,8.7,3.0,100.0,-5.0,5.0,8.0,...,16.0,44.0,0.0,1028.0,10.0,10.0,291.0,19.0,309.0,Hazardous
2500,2021-01-29,9.0,6.0,0.0,8.7,3.0,90.0,-9.0,5.0,7.0,...,0.0,32.0,0.0,1030.0,9.0,10.0,188.0,10.0,320.0,Hazardous


## task definition
given weather data, predict AQI

## build complex features:
1. time window(temp)
+ past 1-day avg/high
+ past 3 days avg/high
+ past 7 days avg/high

In [63]:
DEFAULT_FEATURE_VALUE = -999.0

build time window features

In [64]:
df_raw['tempC_avg_today'] = df_raw.p_apply(lambda l: (l.maxtempC - l.mintempC) / 2, axis=1)

<LAMBDA> DONE:   0%|          | 0/2502 [00:00<?, ?it/s]

In [65]:
def set_time_window_avg_feature(df, time_window: int, window_col_name: str, base_col_name: str):
    df[window_col_name] = DEFAULT_FEATURE_VALUE  # initialize
    for i in range(time_window, len(df)):
        df.at[i, window_col_name] = sum(df[base_col_name].iloc[i - time_window:i].tolist()) / time_window
    return df


def set_time_window_feature(df, time_window: int, window_col_name: str, base_col_name: str):
    df[window_col_name] = DEFAULT_FEATURE_VALUE  # initialize
    for i in range(time_window, len(df)):
        df.at[i, window_col_name] = df[base_col_name].iloc[i - time_window]
    return df

avg temperature

In [66]:
df_raw = set_time_window_avg_feature(df_raw, 1, 'tempC_avg_1days', 'tempC_avg_today')
df_raw = set_time_window_avg_feature(df_raw, 3, 'tempC_avg_3days', 'tempC_avg_today')
df_raw = set_time_window_avg_feature(df_raw, 7, 'tempC_avg_7days', 'tempC_avg_today')

high temperature

In [67]:
df_raw = set_time_window_feature(df_raw, 1, 'tempC_high_1days', 'maxtempC')
df_raw = set_time_window_feature(df_raw, 3, 'tempC_high_3days', 'maxtempC')
df_raw = set_time_window_feature(df_raw, 7, 'tempC_high_7days', 'maxtempC')

low temperature

In [68]:
df_raw = set_time_window_feature(df_raw, 1, 'tempC_low_1days', 'mintempC')
df_raw = set_time_window_feature(df_raw, 3, 'tempC_low_3days', 'mintempC')
df_raw = set_time_window_feature(df_raw, 7, 'tempC_low_7days', 'mintempC')

2. time feature
+ month

In [69]:
df_raw['month'] = df_raw.p_apply(lambda l: int(l.date[5:7]), axis=1)

<LAMBDA> DONE:   0%|          | 0/2502 [00:00<?, ?it/s]

In [70]:
df_raw = df_raw[6:]

## filter feature

In [71]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2496 entries, 6 to 2501
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               2496 non-null   object 
 1   maxtempC           2496 non-null   float64
 2   mintempC           2496 non-null   float64
 3   totalSnow_cm       2496 non-null   float64
 4   sunHour            2496 non-null   float64
 5   uvIndex            2496 non-null   float64
 6   moon_illumination  2496 non-null   float64
 7   DewPointC          2496 non-null   float64
 8   FeelsLikeC         2496 non-null   float64
 9   HeatIndexC         2496 non-null   float64
 10  WindChillC         2496 non-null   float64
 11  WindGustKmph       2496 non-null   float64
 12  cloudcover         2496 non-null   float64
 13  humidity           2496 non-null   float64
 14  precipMM           2496 non-null   float64
 15  pressure           2496 non-null   float64
 16  tempC              2496 

In [72]:
df_raw.drop(['date','AQI_Explained'],inplace=True,axis=1)

C:\Users\fyk12\AppData\Local\Temp\ipykernel_14412\374109260.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw.drop(['date','AQI_Explained'],inplace=True,axis=1)


In [73]:
df_raw

,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,...,tempC_avg_1days,tempC_avg_3days,tempC_avg_7days,tempC_high_1days,tempC_high_3days,tempC_high_7days,tempC_low_1days,tempC_low_3days,tempC_low_7days,month
6,12.0,9.0,0.0,5.5,3.0,39.0,8.0,8.0,10.0,8.0,...,3.0,3.166667,-999.000000,11.0,10.0,-999.0,5.0,4.0,-999.0,1
7,9.0,4.0,0.0,4.0,2.0,46.0,5.0,4.0,8.0,4.0,...,1.5,2.666667,3.285714,12.0,10.0,15.0,9.0,3.0,5.0,1
8,5.0,2.0,0.0,8.7,2.0,53.0,-2.0,1.0,4.0,1.0,...,2.5,2.333333,2.928571,9.0,11.0,14.0,4.0,5.0,7.0,1
9,7.0,1.0,0.0,8.7,2.0,61.0,0.0,4.0,6.0,4.0,...,1.5,1.833333,2.642857,5.0,12.0,16.0,2.0,9.0,9.0,1
10,7.0,5.0,0.0,4.0,2.0,68.0,3.0,3.0,6.0,3.0,...,3.0,2.333333,2.571429,7.0,9.0,10.0,1.0,4.0,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2497,12.0,10.0,0.0,4.0,3.0,88.0,7.0,10.0,11.0,10.0,...,2.5,2.166667,2.285714,14.0,9.0,10.0,9.0,7.0,6.0,1
2498,11.0,7.0,0.0,8.7,2.0,96.0,4.0,7.0,9.0,7.0,...,1.0,2.166667,2.142857,12.0,12.0,14.0,10.0,6.0,9.0,1
2499,10.0,4.0,0.0,8.7,3.0,100.0,-5.0,5.0,8.0,5.0,...,2.0,1.833333,2.071429,11.0,14.0,20.0,7.0,9.0,12.0,1
2500,9.0,6.0,0.0,8.7,3.0,90.0,-9.0,5.0,7.0,5.0,...,3.0,2.000000,1.928571,10.0,12.0,12.0,4.0,10.0,10.0,1


## split train and test

In [74]:
import numpy as np

In [75]:
cap_train = int(0.8 * len(df_raw))

In [76]:
df_train = df_raw[:cap_train]
df_test = df_raw[cap_train:]

In [77]:
train_x = np.array(df_train.drop(['AQI'],axis=1))
train_y = np.array(df_train["AQI"])

In [78]:
test_x = np.array(df_test.drop(['AQI'],axis=1))
test_y = np.array(df_test["AQI"])

# modeling

In [79]:
from lightgbm import LGBMRegressor

In [98]:
model = LGBMRegressor(num_leaves=2**10-1)
model.fit(train_x, train_y)

LGBMRegressor(num_leaves=1023)

In [99]:
out = model.predict(test_x)

In [100]:
abs(test_y - out).mean()

60.200419424751146